# `nbgallery` Bulk Uploader

Quick hack to automate the uploading of notebooks to `nbgallery`.

Uses `selenium` (I couldn't get `mechanize` / `mechanical soup` to work?).

In [179]:
from selenium import webdriver

#Selenium package includes several utilitities
# for waiting until things are ready
#https://selenium-python.readthedocs.io/waits.html
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC


In [ ]:
driver = webdriver.Chrome()

#Allow the driver to poll the DOM for up to 10s when
# trying to find an element
driver.implicitly_wait(10)

#We might also want to explicitly define wait conditions
# on a particular element
wait = WebDriverWait(driver, 10)

driver.get("http://localhost:3000/")

In [178]:
#Close the browser with:
#driver.close()

## Login

In [130]:
NBGALLERY_EMAIL=''
NBGALLERY_PASSWORD=''

In [131]:
driver.find_element_by_id("gearDropdown").click()

element = driver.find_element_by_id("user_email")

In [132]:
element.click()
element.clear()
element.send_keys(NBGALLERY_EMAIL)

In [133]:
element = driver.find_element_by_id("user_password")
element.clear()
element.send_keys(NBGALLERY_PASSWORD)
element.click()

In [134]:
driver.find_element_by_xpath("//input[@value='Login']").click()

As a function...

In [195]:
def nbgallery_login(driver, wait, user, pwd):
    ''' Login to nbgallery.
        Return once the login dialogue has disappeared.
    '''
    
    driver.find_element_by_id("gearDropdown").click()
    
    element = driver.find_element_by_id("user_email")
    element.click()
    
    element.clear()
    element.send_keys(user)
    
    element = driver.find_element_by_id("user_password")
    element.clear()
    element.send_keys(pwd)
    element.click()
    
    driver.find_element_by_xpath("//input[@value='Login']").click()
    
    #Wait until the login has been accepted
    #May be better to check for presence of logged in indicator?
    #A crude way would be to test for presence of /Logout/ in page
    #Alternatively, is a new page is loaded following a login?
    wait.until(EC.invisibility_of_element_located((By.ID, 'user_email')))
    

## Upload Part 1

The upload is a two part process.

In the first part, we identify the notebook file we want to upload.

<div><img src='images/nbgallery1.png' style="height: 400px; padding: 20px" /></div>

The acknowledgements box needs to be checked in order to proceed.

Note that a check in the page tests that the file has an `.ipynb` suffix, so we might also want to check for that too...

In [135]:
driver.find_element_by_id("uploadModalButton").click()

In [136]:
driver.find_element_by_id("uploadFile").send_keys("/Users/tonyhirst/notebooks/Untitled.ipynb");

In [137]:
driver.find_element_by_xpath('//*[@id="uploadFileForm"]/div[3]/div/div/label/input').click()

In [138]:
driver.find_element_by_id("uploadFileSubmit").click()

## Upload Part 2

The second part of the upload process involves submitting compulsoty non-null *title* and *description* values. Again, the acknowledgements box needs to be checked in order to proceed.

<div><img src='images/nbgallery2.png' style="height: 400px; padding: 20px" /></div>


In [139]:
element = driver.find_element_by_id("stageTitle")
element.click()
element.send_keys("My title")

In [140]:
element = driver.find_element_by_id("stageDescription")
element.click()
element.send_keys("My description")

In [141]:
element = driver.find_element_by_id("stageTags-tokenfield")
element.click()
element.send_keys("tag1,tag2,") #need the final comma to set it?

In [142]:
#optional
#driver.find_element_by_id("stagePrivate").click()

In [143]:
driver.find_element_by_xpath('//*[@id="stageForm"]/div[9]/div/div/label/input').click()

In [144]:
driver.find_element_by_id("stageSubmit").click()

If it's a duplicate notebook, we get an error page, with an extra item in the form that gives us the option to overwrite the original file:

`<input name="overwrite" type="checkbox" value="true">`

Should try to check for this somehow? What would trigger the check?

In [ ]:
driver.close()

## Bulk Uploader

Let's put the pieces together and create a bulk uploader...

In [202]:

def nbgallery_upload(driver, wait,
                     path, title='', desc='', tags=None, private=False):
    ''' Upload a notebook.
        Return once the notebook page has loaded.
    '''

    #path is full path to file
    if not path.endswith('.ipynb'):
        print('Not a notebook (.ipynb) file? [{}]'.format(path))
        return
    
    #Part 1
    
    element = wait.until(EC.element_to_be_clickable((By.ID, 'uploadModalButton')))
    element.click()
    
    driver.find_element_by_id("uploadFile").send_keys(path);
    driver.find_element_by_xpath('//*[@id="uploadFileForm"]/div[3]/div/div/label/input').click()
    driver.find_element_by_id("uploadFileSubmit").click()
    
    
    #Part 2
    element = driver.find_element_by_id("stageTitle")
    element.click()
    
    
    #Is there notebook metadata we can search for title?
    if not title:
        title = path.split('/')[-1].replace('.ipynb','')
    element.clear()
    element.send_keys(title)
    
    element = driver.find_element_by_id("stageDescription")
    element.click()
    
    #Is there notebook metadata we can search for description?
    #Any other notebook metadata we could make use of here?
    element.clear()
    #Description needs to be not null
    desc= 'No description.' if not desc else desc
    element.send_keys(desc)
    
    element = driver.find_element_by_id("stageTags-tokenfield")
    element.click()
    
    #Handle various tagging styles
    #Is there notebook metadata we can search for tags?
    tags = '' if not tags else tags
    if isinstance(tags, list):
        tags=','.join(tags)
    tags = tags if tags.endswith(',') else tags+','
    
    element.clear()
    element.send_keys(tags) #need the final comma to set it?
    
    if private:
        driver.find_element_by_id("stagePrivate").click()
        
    driver.find_element_by_xpath('//*[@id="stageForm"]/div[9]/div/div/label/input').click()
    driver.find_element_by_id("stageSubmit").click()
    
    #https://blog.codeship.com/get-selenium-to-wait-for-page-load/
    #Wait for new page to load
    wait.until(EC.staleness_of(driver.find_element_by_tag_name('html')))


## Step wise check...

Do the steps work?

In [165]:
#Open browser
driver = webdriver.Chrome()
driver.implicitly_wait(10)
wait = WebDriverWait(driver, 10)

driver.get("http://localhost:3000/")

In [166]:
#Login
nbgallery_login(driver, wait, NBGALLERY_EMAIL, NBGALLERY_PASSWORD)

In [170]:
#Upload two notebooks, one after the other
nbgallery_upload(driver, '/Users/tonyhirst/notebooks/nbgallery scraper.ipynb')
nbgallery_upload(driver, '/Users/tonyhirst/notebooks/Untitled2.ipynb')


In [176]:
#Close browser
driver.close()

## Full Check

Do the steps work when run automatically, rather than manually stepped through?

In [204]:
#Open browser
driver = webdriver.Chrome()
driver.implicitly_wait(10)
wait = WebDriverWait(driver, 10)

driver.get("http://localhost:3000/")

#Login
nbgallery_login(driver, wait, NBGALLERY_EMAIL, NBGALLERY_PASSWORD)

#Upload two notebooks, one after the other
nbgallery_upload(driver, wait, '/Users/tonyhirst/notebooks/Untitled.ipynb')
nbgallery_upload(driver, wait, '/Users/tonyhirst/notebooks/Untitled2.ipynb')

#Close browser
driver.close()